# Week 10 Assignment: CVA Data Exploration 
# Group Name: Extreme Heat in Los Angeles
This week, I will attempt to explore the CVA vulnerability data for the final and create some sort of index.
Author: Amanda Gormsen

In [5]:
# to manipulate data
import pandas as pd

# for spatial data
import geopandas as gpd

# for plotting
import matplotlib.pyplot as plt

# for network analysis
import networkx as nx

# for street network analysis
import osmnx as ox

# for basemaps
import contextily as ctx

# for interactive webmaps
import folium

In [6]:
!pip install pysal

  Using cached pysal-23.1-py3-none-any.whl (17 kB)
  Using cached spint-1.0.7-py3-none-any.whl
  Using cached spglm-1.0.8-py3-none-any.whl
  Using cached mgwr-2.1.2-py3-none-any.whl
  Using cached tobler-0.9.0-py3-none-any.whl (22 kB)
  Using cached segregation-2.3.1-py3-none-any.whl (147 kB)
  Using cached pointpats-2.3.0-py3-none-any.whl (59 kB)
  Using cached momepy-0.5.4-py3-none-any.whl (243 kB)
  Using cached spopt-0.5.0-py3-none-any.whl (112 kB)
  Using cached access-1.1.8-py3-none-any.whl (21 kB)
  Using cached spvcm-0.3.0-py3-none-any.whl
  Using cached inequality-1.0.0-py3-none-any.whl
  Using cached spaghetti-1.7.2-py3-none-any.whl (50 kB)
  Using cached pygeos-0.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached rvlib-0.0.6-cp310-cp310-linux_x86_64.whl
  Using cached quilt3-5.1.1-py3-none-any.whl (79 kB)
  Using cached PuLP-2.7.0-py3-none-any.whl (14.3 MB)
  Using cached rasters

In [7]:
# Importing pysal to reclassify data
import pysal as ps
import mapclassify
n_classes = 5

In [8]:
# Importing data
df=gpd.read_file('Los_Angeles_County_CVA_Social_Vulnerability_Index.geojson') 

These are the variables that the CVA dataset considers for social vulnerability: 

Children
Older_Adults
Older_Adults_Living_Alone
Limited_English
No_High_School_Diploma
Female
Female_Householder
Asthma
Cardiovascular_Disease
Disability
No_Health_Insurance
Living_in_Group_Quarters
Mobile_Homes
Rent_Burden
Renters
Median_Income
Poverty
Households_Without_Vehicle_Acce
Transit_Access
Outdoor_Workers
Unemployed
Foreign_Born
Library_Access
No_Internet_Subscription
Voter_Turnout_Rate
Hispanic_Latinx
White
Black
Native_Tribal
Asian
NHOPI
Other_Race
Two_or_More_Races
SoVI_Score
SoVI_Thirds

I want to focus on the variables that seem most related to extreme heat vulnerability, esp related to cooling centers and transit. These are the variables I will use in my custom index:
- Poverty (values = 0 to 83)
- Households_Without_Vehicle_Acce (values = 0 to 85)
- Transit_Access (values = 0 to 100)
- Outdoor_Workers (values = 0 to 32)
- Library_Access (values = 0 to 4)

In [19]:
# Trimming data
df_trimmed=df[['Census_Tract','Poverty', 'Households_Without_Vehicle_Acce', 'Transit_Access', 'Outdoor_Workers', 'Library_Access','geometry']]

In [20]:
classifier=mapclassify.NaturalBreaks.make(k=5)

In [21]:
df_trimmed['poverty_classes']=df_trimmed[["Poverty"]].apply(classifier)
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1
...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1


In [22]:
df_trimmed['vehicleaccess_classes']=df_trimmed[["Households_Without_Vehicle_Acce"]].apply(classifier)
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes,vehicleaccess_classes
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0,0
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0,1
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1,0
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0,0
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1,1
...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2,2
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2,0
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0,0
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1,1


In [23]:
df_trimmed['transitaccess_classes']=df_trimmed[["Transit_Access"]].apply(classifier)
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes,vehicleaccess_classes,transitaccess_classes
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0,0,0
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0,1,4
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1,0,0
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0,0,0
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1,1,0
...,...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2,2,4
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2,0,0
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0,0,0
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1,1,3


In [24]:
df_trimmed['outdoorwork_classes']=df_trimmed[["Outdoor_Workers"]].apply(classifier)
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes,vehicleaccess_classes,transitaccess_classes,outdoorwork_classes
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0,0,0,1
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0,1,4,0
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1,0,0,3
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0,0,0,1
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2,2,4,2
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2,0,0,2
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0,0,0,0
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1,1,3,2


In [25]:
df_trimmed['libraryaccess_classes']=df_trimmed[["Library_Access"]].apply(classifier)
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes,vehicleaccess_classes,transitaccess_classes,outdoorwork_classes,libraryaccess_classes
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0,0,0,1,0
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0,1,4,0,1
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1,0,0,3,2
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0,0,0,1,3
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2,2,4,2,0
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2,0,0,2,1
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0,0,0,0,1
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1,1,3,2,0


In [26]:
df_trimmed['Heat_Score']=df_trimmed['poverty_classes']+df_trimmed['vehicleaccess_classes']+df_trimmed['transitaccess_classes']+df_trimmed['outdoorwork_classes']+df_trimmed['libraryaccess_classes']
df_trimmed

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Census_Tract,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Library_Access,geometry,poverty_classes,vehicleaccess_classes,transitaccess_classes,outdoorwork_classes,libraryaccess_classes,Heat_Score
0,Census Tract 5709.02,1.5,3.8,0.0,5.6,0.8,"POLYGON ((-118.12509 33.86032, -118.12287 33.8...",0,0,0,1,0,1
1,Census Tract 5715.02,10.0,9.0,98.3,2.5,1.2,"POLYGON ((-118.19274 33.84376, -118.18975 33.8...",0,1,4,0,1,6
2,Census Tract 9106.05,17.1,3.3,0.0,11.6,2.9,"POLYGON ((-118.07616 34.57281, -118.06728 34.5...",1,0,0,3,2,6
3,Census Tract 9107.12,7.0,0.9,0.0,3.6,4.2,"POLYGON ((-118.05857 34.55809, -118.05847 34.5...",0,0,0,1,3,4
4,Census Tract 9107.15,11.3,8.9,0.0,4.6,3.1,"POLYGON ((-118.08506 34.55654, -118.08485 34.5...",1,1,0,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,19.6,12.4,100.0,7.1,0.7,"POLYGON ((-118.35263 33.95271, -118.35208 33.9...",2,2,4,2,0,10
2323,Census Tract 9005.01,28.7,4.9,0.0,7.3,1.5,"POLYGON ((-118.13480 34.68936, -118.12968 34.6...",2,0,0,2,1,5
2324,Census Tract 9200.29,5.4,3.7,0.0,2.7,1.6,"POLYGON ((-118.50373 34.42607, -118.50050 34.4...",0,0,0,0,1,1
2325,Census Tract 5420,13.3,8.7,88.0,9.3,0.7,"POLYGON ((-118.20731 33.90754, -118.20641 33.9...",1,1,3,2,0,7


In [27]:
# Mapping
m=folium.Map(location=[34.2,-118.2],
             zoom_start=9,
             tiles='CartoDB positron',
             attribute='CartoDB')

In [28]:
folium.Choropleth(
geo_data=df_trimmed,
    data=df_trimmed,
    key_on='feature.properties.Census_Tract',
    columns=['Census_Tract', 'Heat_Score'],
    fill_color='BuPu',
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Heat Vulnerability Index').add_to(m)
m

In [31]:
#Make this Notebook Trusted (File>Trust Notebook)
jupyter trust /up221-amanda/Week07/week 10 assignment.ipynb

SyntaxError: invalid syntax (1680040479.py, line 2)